In [ ]:
import os
import glob

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from skimage.io import imread, imsave

In [ ]:
short_codes = [
    'Background',
    'LC_Fol', 
    'LC_Branch', 
    'Ar_TA', 
    'LC_Por', 
    'UDC_CCA', 
    'Ro_TA', 
    'Pa_FA', 
    'Mu_Ba', 
    'CR_TA', 
    'Pa_Cy', 
    'LC_Encr', 
    'Sa_Ba', 
    'Pa_H', 
    'Pa_TA', 
    'UDC_H', 
    'CR_FA', 
    'Ro_CCA', 
    'Mu_Cy', 
    'UDC_TA', 
    'Sa_Cy', 
    'UDC_FA', 
    'Ro_H', 
    'Sa_FA', 
    'Pa_CCA', 
    'Ro_Ba', 
    'Ro_Cy', 
    'Sa_TA', 
    'Ar_Ba', 
    'UDC_Cy', 
    'Ar_FA', 
    'CR_CCA'
]

# Creating a mapping of short codes to color values
class2label = {k:i for i, k in enumerate(short_codes)}

In [ ]:
data_root = "/home/azureuser/cloudfiles/code/Users/jordan.pierce/Data/Guam_Saipan/3653/"
image_path = data_root + "images/"
mask_path = data_root + "masks/"
label_path = data_root + "CNet_Annotations.csv"

os.makedirs(mask_path, exist_ok=True)

df = pd.read_csv(label_path)
df = df[df['Annotator'] == 'Imported']

In [ ]:
def label_mask(mask, y, x, class_category, size=30):
    """Given an image, and a Y, X location, this function will extract the patch. 
        Default patch size is 112 x 112"""

    mask[abs(size - y) : abs(size + y), abs(size - x) : abs(size + x)] = class_category
    
    return mask

    
mask_df = []

for image_name in df['Name'].unique():

    image_name = image_path + image_name
    mask_name = mask_path + os.path.basename(image_name).split(".")[0] + ".npy"

    width, height = Image.open(image_name).size
    mask = np.zeros(shape = (height, width), dtype=np.uint8)

    image_df = df[df['Name'] == os.path.basename(image_name)]
    
    for i, r in image_df.iterrows():

        y, x = r['Row'], r['Column']
        class_category = class2label[r['Label']]
        mask = label_mask(mask, y, x, class_category, 112)

    try:
        
        np.save(file=mask_name, arr=mask)
        mask_df.append([image_name, mask_name])

    except Exception as e:
        print(e)
        pass 
       
mask_df = pd.DataFrame(mask_df, columns=['Image', 'Mask'])
mask_df.to_csv(data_root + "Updated_CNet_Segmentation_Masks.csv")